In [1]:
import googleapiclient.discovery
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
import re
from googleapiclient.errors import HttpError

In [2]:
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

mycursor.execute('drop database if exists youtube')
api_service_name = "youtube"
api_version = "v3"
api_key ="************"

In [3]:
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = api_key)

In [4]:
def channel_data_df(channel_id):
    request = youtube.channels().list(
            part="contentDetails,snippet,statistics",
            id= channel_id
        )
    response = request.execute()

    data={ 'channel_id':channel_id,
            'channel_name':response['items'][0]['snippet']['title'], 
            'channel_des':response['items'][0]['snippet']['description'],
            'channel_playid':response['items'][0]['contentDetails']['relatedPlaylists']['uploads'],
            'channel_vidcount':response['items'][0]['statistics']['videoCount'],
            'channel_viewcount':response['items'][0]['statistics']['viewCount'],
            'channel_subcount':response['items'][0]['statistics']['subscriberCount']}
            
    df=pd.DataFrame(data,index=[0])

    return df

In [5]:
def allchanneldetails(channel_ids):
    mydb = mysql.connector.connect(host="localhost",user="root",password="")

    mycursor = mydb.cursor(buffered=True)

    db_connection_str = f"mysql+mysqlconnector://root@localhost/youtube"
    db_engine = create_engine(db_connection_str)

    mycursor.execute('create database if not exists youtube')
    mycursor.execute('USE youtube')
    mycursor.execute('''create table if not exists channels(channel_id VARCHAR(50), 
                    channel_name VARCHAR(100),channel_des TEXT, channel_playid VARCHAR(40), 
                    channel_vidcount INT(5), channel_viewcount INT(10), channel_subcount INT(10))''')
    mydb.commit()
    for channel_id in channel_ids:
        df=channel_data_df(channel_id)
        df.to_sql(name='channels', con=db_engine, if_exists='append', index=False)

    mycursor.close()
    mydb.close()

channel_ids = ['UCduIoIMfD8tT3KoU0-zBRgQ',
'UC7CKJFP4Tzd2n6Zb-Q4P25Q',
'UCmZq4z5pEnljTjGf2f6kMRw',
'UCkLBvNwaytweHytgv1P8dhg',
'UC3uyFWA21nRwGEuGT6hg3pQ',
'UCaMW02R76uFbo--dS24oBOg',
'UCvBZ97Qkj7tyTr9RZsEeKhg',
'UCpVm7bg6pXKo1Pr6k5kxG9A',
'UCetJvfhHNboF_Tl3auMOs5w',
'UCXXYoChpS5hLeZrTaI2I2Mw']
allchanneldetails(channel_ids)

In [6]:
def iso8601_duration_to_seconds(duration):
    match = re.match(r'^PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?$', duration)
    if not match:
        return None

    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0

    total_seconds = (hours * 3600) + (minutes * 60) + seconds
    return total_seconds

In [7]:
def video_details_info_df(video_ids):
    video_data=[]
    for video_id in video_ids:
        request = youtube.videos().list(
                part="contentDetails,snippet,statistics",
                id=video_id
            )
        response = request.execute()

        for details in response["items"]:
            caption_available = details['contentDetails'].get('caption', False)
            data= {
                'Video_Id': details['id'],
                'Video_title': details['snippet']['title'],
                'channel_id': details['snippet']['channelId'],
                'Video_Description': details['snippet']['description'],
                'Video_pubdate': details['snippet']['publishedAt'],
                'Video_thumbnails': details['snippet']['thumbnails']['default']['url'],
                'Video_viewcount': details['statistics']['viewCount'],
                'Video_likecount': details['statistics'].get('likeCount', 0),
                'Video_favoritecount': details['statistics']['favoriteCount'],
                'Video_commentcount': details['statistics'].get('commentCount', 0),
                'Video_duration': iso8601_duration_to_seconds(details['contentDetails']['duration']),
                'Video_caption': caption_available
            }

            video_data.append(data)

    df1 = pd.DataFrame(video_data)

    return df1
video_ids = ['rtNOm77yM0I', 'HvB7i6Gzj2E',
                    'zeKPwCCi00k&t=1s', 'asAdN-ToO7c',
                    'ize4a2XZmP4' , 'nHWbYD0fLHA',
                    'wtoeyN_Y63Q','jZKccUE2poo',
                    '65Wt5XNBWSc','nbfDX-KOHWc',
                    'RubYPMDOzus', 'bcclDbdbgKA',
                    'mnQ2YT1DZrw', 'PE0q76vg-FE',
                    'd3b6zSpy7P4','InQMoBvzQeY',
                    'sBhYE4JI030' ,  'KPLDUK8xUnQ', 
                    '6MKDzp_a9d4','RNQmtrhWCzY']

df1 = video_details_info_df(video_ids)

mydb = mysql.connector.connect(host="localhost",user="root",password="")

mycursor = mydb.cursor(buffered=True)

db_connection_str = f"mysql+mysqlconnector://root@localhost/youtube"
db_engine = create_engine(db_connection_str)

mycursor.execute('use youtube')
mycursor.execute('''create table if not exists videos (Video_Id VARCHAR(50),
                Video_title TEXT, channel_id VARCHAR(50), Video_Description TEXT, Video_pubdate VARCHAR(30),
                Video_thumbnails TEXT, Video_viewcount INT(15), Video_likecount INT(15),
                Video_favoritecount INT(15), Video_commentcount INT(15),
                Video_duration VARCHAR(10), Video_caption VARCHAR(10)
                ) ''')

mydb.commit()

df1.to_sql(name='videos', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()

In [8]:
def comment_details_info_df(video_ids):
    comment_data = []
    try:
        for video_id in video_ids:
            next_page_token = None
            while True:
                try:
                    request_comments = youtube.commentThreads().list(
                        part="snippet",
                        videoId=video_id,
                        maxResults=100,
                        pageToken=next_page_token)
                    response_comments = request_comments.execute()

                    for comment in response_comments["items"]:
                        data = {
                            'comment_id': comment['snippet']['topLevelComment']['id'],
                            'video_id': comment['snippet']['topLevelComment']['snippet']['videoId'],
                            'channel_id': comment['snippet']['channelId'],
                            'author_name': comment['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                            'text_display': comment['snippet']['topLevelComment']['snippet']['textDisplay'],
                            'published_date': comment['snippet']['topLevelComment']['snippet']['publishedAt']
                        }
                        comment_data.append(data)

                    next_page_token = response_comments.get('nextPageToken')

                    if next_page_token is None:
                        break
                except HttpError as e:
                    if e.resp.status == 404:
                        print(f"Comments are disabled for video ID: {video_id}")
                        break
                    else:
                        raise
    except Exception as e:
        print(f"An error occurred: {e}")

    df2 = pd.DataFrame(comment_data)
    return df2

video_ids = ['rtNOm77yM0I', 'HvB7i6Gzj2E',
                    'zeKPwCCi00k&t=1s', 'asAdN-ToO7c',
                    'ize4a2XZmP4' , 'nHWbYD0fLHA',
                    'wtoeyN_Y63Q','jZKccUE2poo',
                    '65Wt5XNBWSc','nbfDX-KOHWc',
                    'RubYPMDOzus', 'bcclDbdbgKA',
                    'mnQ2YT1DZrw', 'PE0q76vg-FE',
                    'd3b6zSpy7P4','InQMoBvzQeY',
                    'sBhYE4JI030' ,  'KPLDUK8xUnQ', 
                    '6MKDzp_a9d4','RNQmtrhWCzY']

df2 = comment_details_info_df(video_ids)

mydb = mysql.connector.connect(host="localhost",user="root",password="")

mycursor = mydb.cursor(buffered=True)

db_connection_str = f"mysql+mysqlconnector://root@localhost/youtube"
db_engine = create_engine(db_connection_str)

mycursor.execute('use youtube')
mycursor.execute('''create table if not exists comments (comment_id VARCHAR(30), video_id VARCHAR(15),
                channel_id VARCHAR(50),author_name LONGTEXT, text_display TEXT, published_date VARCHAR(20)
                ) ''')

mydb.commit()

df2.to_sql(name='comments', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()

Comments are disabled for video ID: zeKPwCCi00k&t=1s


In [9]:
#1 .What are the names of all the videos and their corresponding channels?

import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT videos.Video_title, channels.channel_name
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------------------------------------------------------------------------------------------+-------------------------------------------------+
| Video_title                                                                                          | channel_name                                    |
|------------------------------------------------------------------------------------------------------+-------------------------------------------------|
| MONEYBALL Movie - Data Science Strategies Revealed | Moneyball Movie Review|Tech Movie Review | GUVI | GUVI                                            |
| ChatGPT-5: What to Expect? | Top 10 Tech News | Apple | chatGPT 5 | Microsoft | Meta | AI | GUVI     | GUVI                                            |
| How to crack the engineering job placement? Every student must watch #interviewhacks #SREC           | Sri Ramakrishna Engineering College             |
| The 10 Commandments for our Beginner Students | Israel Jebasingh | O

In [10]:
#2 .Which channels have the most number of videos, and how many videos do they have?

import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT channel_name, COUNT(*) AS video_count
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
GROUP BY channel_name
ORDER BY video_count DESC
LIMIT 1;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-------------------------+---------------+
| channel_name            |   video_count |
|-------------------------+---------------|
| Israel Jebasingh Ex IAS |             3 |
+-------------------------+---------------+


In [11]:
#3. What are the top 10 most viewed videos and their respective channels?

import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT videos.Video_title, channels.channel_name
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
ORDER BY videos.Video_viewcount DESC
LIMIT 10;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+---------------------------------------------------------------------------------------------+-------------------------------------------------+
| Video_title                                                                                 | channel_name                                    |
|---------------------------------------------------------------------------------------------+-------------------------------------------------|
| 2023 | Another Extraordinary Year for Rolls-Royce                                           | Rolls-Royce Motor Cars                          |
| The Caterpillar Trail | Diggin' Into History                                                | Caterpillar Inc.                                |
| Dr Sara and her Black Badge Ghost | The Spirit of Rolls-Royce Episode 9                     | Rolls-Royce Motor Cars                          |
| How﻿ to do IAS/IPS... Exam Self Preparation during College? | Israel Jebasingh | Tamil       | Israel Jebasingh Ex IAS    

In [12]:
#4. How many comments were made on each video, and what are their corresponding video names?

import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT videos.Video_title, COUNT(*) AS comment_count
FROM videos
JOIN comments ON videos.Video_Id = comments.video_id
GROUP BY videos.Video_title;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------------------------------------------------------------------------------------------+-----------------+
| Video_title                                                                                          |   comment_count |
|------------------------------------------------------------------------------------------------------+-----------------|
| 1st Attempt-ல் IAS தேர்ச்சி in just 8 months | Prashanth IAS | AIR 78 | Israel Jebasingh | Tamil          |              65 |
| 2023 | Another Extraordinary Year for Rolls-Royce                                                    |             109 |
| Antique Ice Cream Maker Restoration                                                                  |            2614 |
| Antique Lighter Restoration. Over 100 years Underground!                                             |              33 |
| Caterpillar's Contribution to World War II | Diggin' into History                                    |               8 |
| ChatGPT-5

In [13]:
#5. Which videos have the highest number of likes, and what are their  corresponding channel names?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""

SELECT videos.Video_title, channels.channel_name
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
ORDER BY videos.Video_likecount DESC
LIMIT 1;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+---------------------------------------------------+------------------------+
| Video_title                                       | channel_name           |
|---------------------------------------------------+------------------------|
| 2023 | Another Extraordinary Year for Rolls-Royce | Rolls-Royce Motor Cars |
+---------------------------------------------------+------------------------+


In [14]:
#6. What is the total number of likes for each video, and what are  their corresponding video names?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT videos.Video_title, SUM(videos.Video_likecount) AS total_likes
FROM videos
GROUP BY videos.Video_title;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------------------------------------------------------------------------------------------+---------------+
| Video_title                                                                                          |   total_likes |
|------------------------------------------------------------------------------------------------------+---------------|
| 1st Attempt-ல் IAS தேர்ச்சி in just 8 months | Prashanth IAS | AIR 78 | Israel Jebasingh | Tamil          |          1260 |
| 2023 | Another Extraordinary Year for Rolls-Royce                                                    |          4011 |
| Antique Ice Cream Maker Restoration                                                                  |        104330 |
| Antique Lighter Restoration. Over 100 years Underground!                                             |           896 |
| Caterpillar's Contribution to World War II | Diggin' into History                                    |            90 |
| ChatGPT-5: What to Expect

In [15]:
#7. What is the total number of views for each channel, and what are their  corresponding channel names?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT channels.channel_name, SUM(videos.Video_viewcount) AS total_views
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
GROUP BY channels.channel_name;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-------------------------------------------------+---------------+
| channel_name                                    |   total_views |
|-------------------------------------------------+---------------|
| Caterpillar Inc.                                |        129406 |
| GUVI                                            |          5088 |
| Israel Jebasingh Ex IAS                         |        118691 |
| John Victor IAS teacher                         |          2185 |
| National Geographic                             |         34007 |
| Officers IAS Academy - India's Only IAS Academy |         21757 |
| Polity - Rangarajan Ex IAS                      |          3339 |
| ReXtorer                                        |         13243 |
| Rolls-Royce Motor Cars                          |        289374 |
| Sri Ramakrishna Engineering College             |          3920 |
+-------------------------------------------------+---------------+


In [16]:
#8. What are the names of all the channels that have published videos in the year 2022?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT DISTINCT channels.channel_name
FROM channels
JOIN videos ON channels.channel_id = videos.channel_id
WHERE YEAR(videos.Video_pubdate) = 2022;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-------------------------------------------------+
| channel_name                                    |
|-------------------------------------------------|
| Sri Ramakrishna Engineering College             |
| Officers IAS Academy - India's Only IAS Academy |
+-------------------------------------------------+


In [17]:
#9. What is the average duration of all videos in each channel, and what are their corresponding channel names?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT channels.channel_name, AVG(videos.Video_duration) AS average_duration
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
GROUP BY channels.channel_name;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-------------------------------------------------+--------------------+
| channel_name                                    |   average_duration |
|-------------------------------------------------+--------------------|
| Caterpillar Inc.                                |              538.5 |
| GUVI                                            |              762   |
| Israel Jebasingh Ex IAS                         |              885   |
| John Victor IAS teacher                         |              671.5 |
| National Geographic                             |             2735.5 |
| Officers IAS Academy - India's Only IAS Academy |              833   |
| Polity - Rangarajan Ex IAS                      |              562.5 |
| ReXtorer                                        |              839   |
| Rolls-Royce Motor Cars                          |              140.5 |
| Sri Ramakrishna Engineering College             |               61   |
+-------------------------------------------------+

In [18]:
#10. Which videos have the highest number of comments, and what are their corresponding channel names?
import mysql.connector

mydb = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = mydb.cursor()

mycursor.execute('USE youtube')
mycursor.execute("""
SELECT videos.Video_title, channels.channel_name
FROM videos
JOIN channels ON videos.channel_id = channels.channel_id
ORDER BY videos.Video_commentcount DESC
LIMIT 1;
""")

data = mycursor.fetchall()

from tabulate import tabulate
print(tabulate(data,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+---------------------------------------------------------------------------------------+-------------------------+
| Video_title                                                                           | channel_name            |
|---------------------------------------------------------------------------------------+-------------------------|
| How﻿ to do IAS/IPS... Exam Self Preparation during College? | Israel Jebasingh | Tamil | Israel Jebasingh Ex IAS |
+---------------------------------------------------------------------------------------+-------------------------+
